In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import yfinance as yf
import pandas as pd
from datetime import datetime, timedelta
import requests

In [3]:
import sys
print(sys.path)
sys.path.insert(0, "/Users/garciaj42/code/MLFinance/claude")

['/Users/garciaj42/code/MLFinance/claude/notebooks', '/Users/garciaj42/code/MLFinance/claude/notebooks', '/Users/garciaj42/code/MLFinance', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python39.zip', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9', '/opt/homebrew/Cellar/python@3.9/3.9.19_1/Frameworks/Python.framework/Versions/3.9/lib/python3.9/lib-dynload', '', '/opt/homebrew/lib/python3.9/site-packages', '/opt/homebrew/lib/python3.9/site-packages/IPython/extensions', '/opt/homebrew/lib/python3.9/site-packages/setuptools/_vendor', '/Users/garciaj42/.ipython']


In [4]:
lookback_days = 365 * 1

In [27]:
from market_indicator_data import fetch_market_indicators, fill_missing_data, generate_summary_stats, print_summary_stats

# Fetch the data
market_indicators_data, n_markets = fetch_market_indicators(datetime.now() -  timedelta(days=lookback_days))

Len of yfinance.Ticker object <ES=F> = 250
Successfully fetched data for E-mini S&P 500 (ES=F)
Len of yfinance.Ticker object <NQ=F> = 250
Successfully fetched data for E-mini NASDAQ 100 (NQ=F)
Len of yfinance.Ticker object <RTY=F> = 250
Successfully fetched data for E-mini Russell 2000 (RTY=F)
Len of yfinance.Ticker object <YM=F> = 250
Successfully fetched data for E-mini Dow (YM=F)
Len of yfinance.Ticker object <6E=F> = 250
Successfully fetched data for Euro FX (6E=F)
Len of yfinance.Ticker object <6J=F> = 250
Successfully fetched data for Japanese Yen (6J=F)
Len of yfinance.Ticker object <6B=F> = 250
Successfully fetched data for British Pound (6B=F)
Len of yfinance.Ticker object <6A=F> = 250
Successfully fetched data for Australian Dollar (6A=F)
Len of yfinance.Ticker object <6C=F> = 250
Successfully fetched data for Canadian Dollar (6C=F)
Len of yfinance.Ticker object <6M=F> = 250
Successfully fetched data for Mexican Peso (6M=F)
Len of yfinance.Ticker object <6N=F> = 250
Successfu

$GE=F: possibly delisted; no price data found  (1d 2024-02-14 22:41:04.612117 -> 2025-02-13)


Len of yfinance.Ticker object <ZN=F> = 250
Successfully fetched data for 10-Year T-Note (ZN=F)
Len of yfinance.Ticker object <ZT=F> = 250
Successfully fetched data for 2-Year T-Note (ZT=F)
Len of yfinance.Ticker object <ZF=F> = 250
Successfully fetched data for 5-Year T-Note (ZF=F)
Len of yfinance.Ticker object <ZB=F> = 250
Successfully fetched data for 30-Year T-Bond (ZB=F)
Len of yfinance.Ticker object <GE=F> = 0
No data available for Eurodollar (GE=F)


In [26]:
pd.set_option('display.max_rows', 500)
market_indicators_data.tail(500)

,Date,Market,Open,High,Low,Close
7809,2025-01-17 05:00:00+00:00,NG=F,4.280000,4.308000,3.910000,3.948000
7810,2025-01-17 05:00:00+00:00,NQ=F,21227.500000,21682.500000,21209.250000,21594.750000
7811,2025-01-17 05:00:00+00:00,RTY=F,2275.300049,2307.500000,2273.899902,2286.899902
7812,2025-01-17 05:00:00+00:00,SB=F,18.420000,18.459999,18.020000,18.219999
7813,2025-01-17 05:00:00+00:00,SI=F,31.200001,31.209999,30.951000,30.951000
7814,2025-01-17 05:00:00+00:00,SPY,596.960022,599.359985,595.609985,597.580017
7815,2025-01-17 05:00:00+00:00,TLT,87.119172,87.168998,86.730560,86.880028
7816,2025-01-17 05:00:00+00:00,ZC=F,474.000000,485.000000,474.000000,484.250000
7817,2025-01-17 05:00:00+00:00,ZN=F,108.562500,108.859375,108.406250,108.546875
7818,2025-01-17 05:00:00+00:00,ZS=F,1019.000000,1039.000000,1019.000000,1034.000000


In [22]:
market_indicators_data = fill_missing_data(market_indicators_data)

/Users/garciaj42/code/MLFinance/claude/market_indicator_data.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled.loc[mask, col] = df_filled.loc[mask, col].fillna(method='ffill', limit=8)
/Users/garciaj42/code/MLFinance/claude/market_indicator_data.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled.loc[mask, col] = df_filled.loc[mask, col].fillna(method='ffill', limit=8)
/Users/garciaj42/code/MLFinance/claude/market_indicator_data.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will raise in a future version. Use obj.ffill() or obj.bfill() instead.
  df_filled.loc[mask, col] = df_filled.loc[mask, col].fillna(method='ffill', limit=8)
/Users/garciaj42/code/MLFinance/claude/market_indicator_data.py:129: FutureWarning: Series.fillna with 'method' is deprecated and will r

In [23]:
# Generate and print summary statistics
stats_dict = generate_summary_stats(market_indicators_data)

print_summary_stats(market_indicators_data)


=== 6E=F Statistics ===
                     start_date                  end_date  data_points  \
Value 2024-02-12 05:00:00+00:00 2025-02-07 05:00:00+00:00          251   

       missing_pct  mean  std  min  max  
Value          0.0   NaN  NaN  NaN  NaN  



=== 6J=F Statistics ===
                     start_date                  end_date  data_points  \
Value 2024-02-12 05:00:00+00:00 2025-02-07 05:00:00+00:00          251   

       missing_pct  mean  std  min  max  
Value          0.0   NaN  NaN  NaN  NaN  



=== AUDUSD=X Statistics ===
                     start_date                  end_date  data_points  \
Value 2024-02-12 00:00:00+00:00 2025-02-07 00:00:00+00:00          259   

       missing_pct  mean  std  min  max  
Value          0.0   NaN  NaN  NaN  NaN  



=== CADUSD=X Statistics ===
                     start_date                  end_date  data_points  \
Value 2024-02-12 00:00:00+00:00 2025-02-07 00:00:00+00:00          259   

       missing_pct  mean  std  min  ma

In [ ]:
from claude.technical_indicators import multi_indicators

multi_ind = multi_indicators(market_indicators_data, 27)